In [1]:
from datasets import load_dataset

In [2]:
train_ = load_dataset('csv', data_files = '/kaggle/input/nlp-getting-started/train.csv')
test_ = load_dataset('csv', data_files = '/kaggle/input/nlp-getting-started/test.csv')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
train_test_split = train_['train'].train_test_split(test_size = 0.2)

In [4]:
import re

def preprocess(data):
    data['text'] = re.sub(r'@\w+', '', data['text'])
    data['text'] = re.sub(r'http[s]?://\S+', '', data['text'])
#     if data['keyword'] is None:
#         data['keyword'] = 'keyword'
#     data['text'] = ' '.join(data['keyword'].split('%20')) +' [SEP] ' + data['text']
    return data

tr = train_['train'].map(preprocess)

Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [5]:
train = train_test_split['train'].map(preprocess).remove_columns(['id','keyword','location']).rename_column('target', 'label')
val = train_test_split['test'].map(preprocess).remove_columns(['id','keyword','location']).rename_column('target', 'label')
test = test_['train'].map(preprocess).remove_columns(['id','keyword','location'])

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [6]:
train, val, test

(Dataset({
     features: ['text', 'label'],
     num_rows: 6090
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 1523
 }),
 Dataset({
     features: ['text'],
     num_rows: 3263
 }))

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_ckpt = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_new_model():
    return AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = 2).to(device)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
def process(batch):
    return tokenizer(batch['text'], truncation = True, padding = 'max_length', max_length = 32)


train_ds = train.map(process, batched = True)
val_ds = val.map(process, batched = True)
test_ds = test.map(process, batched = True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [9]:
train_ds, val_ds, test_ds

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 6090
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 1523
 }),
 Dataset({
     features: ['text', 'input_ids', 'attention_mask'],
     num_rows: 3263
 }))

In [10]:
import os 

os.environ['WANDB_DISABLED'] = 'true'

In [11]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average = 'weighted') 
    return {'accuracy' : accuracy, 'f1' : f1}

In [12]:
from transformers import TrainingArguments, Trainer

2024-08-23 11:51:04.227614: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 11:51:04.227773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 11:51:04.393679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
output_dir = '/kaggle/working/bert_model'
batch_size = 128

args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 3e-5,
    weight_decay = 0.01,
    num_train_epochs = 4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    logging_steps = 100,
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    overwrite_output_dir = True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
# from peft import LoraConfig, get_peft_model, TaskType


# lora_config = LoraConfig(
#     r = 16,
#     lora_alpha = 64,
#     target_modules = 'all_linear',
#     task_type = 'SEQ_CLS'
# )


# model = get_peft_model(get_new_model(), lora_config)

# trainer = Trainer(
#     model = get_peft_model(get_new_model(), lora_config),
#     args = args,
#     compute_metrics = compute_metrics,
#     train_dataset = train_ds,
#     eval_dataset = val_ds,
#     tokenizer = tokenizer
# )


# trainer.train()

In [15]:
trainer = Trainer(
    model = get_new_model(),
    args = args,
    compute_metrics = compute_metrics,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    tokenizer = tokenizer
)


trainer.train()

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.499967,0.798424,0.795995
2,No log,0.412715,0.833880,0.833229
3,No log,0.391583,0.843729,0.842282
4,No log,0.397888,0.842416,0.841330


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=96, training_loss=0.4304250081380208, metrics={'train_runtime': 304.3727, 'train_samples_per_second': 80.033, 'train_steps_per_second': 0.315, 'total_flos': 1418865500666880.0, 'train_loss': 0.4304250081380208, 'epoch': 4.0})

In [16]:
torch.cuda.empty_cache()

In [17]:
preds = trainer.predict(test_ds)

preds

PredictionOutput(predictions=array([[-2.5694907,  1.445426 ],
       [-2.7590997,  2.4528565],
       [-2.2825928,  2.032712 ],
       ...,
       [-1.521802 ,  1.2894101],
       [-2.4982038,  2.1141968],
       [-2.3253376,  1.9104674]], dtype=float32), label_ids=None, metrics={'test_runtime': 14.6558, 'test_samples_per_second': 222.642, 'test_steps_per_second': 0.887})

In [18]:
x = torch.sigmoid(torch.tensor(preds.predictions)).argmax(-1)

x

tensor([1, 1, 1,  ..., 1, 1, 1])

In [19]:
import pandas as pd

sub = pd.DataFrame({
    'id' : test_['train']['id'],
    'target' : x
}) 


sub.to_csv('submission.csv', index = False)